In [5]:
import copy

In [7]:
spells = [
    {'name': 'Magic missile', 'cost': 53, 'boss_hp': -4},
    {'name': 'Drain', 'cost': 73, 'boss_hp': -2, 'pc_hp': 2},
    {'name': 'Shield', 'cost': 113, 'ongoing': [{'armour': 7}] * 6},
    {'name': 'Poison', 'cost': 173, 'ongoing': [{'boss_hp': -3}] * 6},
    {'name': 'Recharge', 'cost': 229, 'ongoing': [{'mana': 101}] * 5}]
spells

[{'boss_hp': -4, 'cost': 53, 'name': 'Magic missile'},
 {'boss_hp': -2, 'cost': 73, 'name': 'Drain', 'pc_hp': 2},
 {'cost': 113,
  'name': 'Shield',
  'ongoing': [{'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7}]},
 {'cost': 173,
  'name': 'Poison',
  'ongoing': [{'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3}]},
 {'cost': 229,
  'name': 'Recharge',
  'ongoing': [{'mana': 101},
   {'mana': 101},
   {'mana': 101},
   {'mana': 101},
   {'mana': 101}]}]

In [153]:
initial_state = {'pc_hp': 50, 'mana': 500, 'boss_hp': 58, 'boss_damage': 9, 'ongoing': [], 
                 'spent': 0, 'cast': []}

In [154]:
def cast_spell(spell, state):
    new_state = copy.deepcopy(state)
    new_state['mana'] -= spell['cost']
    new_state['spent'] += spell['cost']
    new_state['cast'] += [spell['name']]
    if 'boss_hp' in spell:
        new_state['boss_hp'] += spell['boss_hp']
    if 'pc_hp' in spell:
        new_state['pc_hp'] += spell['pc_hp']
    if 'ongoing' in spell:
        new_state['ongoing'] += [spell['ongoing']]
    return new_state

In [155]:
cast_spell(spells[0], initial_state)

{'boss_damage': 9,
 'boss_hp': 54,
 'cast': ['Magic missile'],
 'mana': 447,
 'ongoing': [],
 'pc_hp': 50,
 'spent': 53}

In [156]:
initial_state

{'boss_damage': 9,
 'boss_hp': 58,
 'cast': [],
 'mana': 500,
 'ongoing': [],
 'pc_hp': 50,
 'spent': 0}

In [157]:
s2 = cast_spell(spells[2], initial_state)
cast_spell(spells[3], s2)

{'boss_damage': 9,
 'boss_hp': 58,
 'cast': ['Shield', 'Poison'],
 'mana': 214,
 'ongoing': [[{'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7}],
  [{'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3}]],
 'pc_hp': 50,
 'spent': 286}

In [97]:
def valid_spells(spells, state):
    valid_spells = []
    for spell in spells:
        add_this_spell = True
        if spell['cost'] > state['mana']:
            add_this_spell = False
        if 'ongoing' in spell:
            for s in spell['ongoing'][0]:
                for status in state['ongoing']:
                    if s in status[0]:
                        add_this_spell = False
        if add_this_spell:
            valid_spells += [spell]
    return valid_spells

In [98]:
valid_spells(spells, initial_state)

[{'boss_hp': -4, 'cost': 53, 'name': 'Magic missile'},
 {'boss_hp': -2, 'cost': 73, 'name': 'Drain', 'pc_hp': 2},
 {'cost': 113,
  'name': 'Shield',
  'ongoing': [{'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7}]},
 {'cost': 173,
  'name': 'Poison',
  'ongoing': [{'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3}]},
 {'cost': 229,
  'name': 'Recharge',
  'ongoing': [{'mana': 101},
   {'mana': 101},
   {'mana': 101},
   {'mana': 101},
   {'mana': 101}]}]

In [158]:
s2 = cast_spell(spells[2], initial_state)
s3 = cast_spell(spells[3], s2)
s3

{'boss_damage': 9,
 'boss_hp': 58,
 'cast': ['Shield', 'Poison'],
 'mana': 214,
 'ongoing': [[{'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7}],
  [{'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3}]],
 'pc_hp': 50,
 'spent': 286}

In [159]:
valid_spells(spells, s3)

[{'boss_hp': -4, 'cost': 53, 'name': 'Magic missile'},
 {'boss_hp': -2, 'cost': 73, 'name': 'Drain', 'pc_hp': 2}]

In [48]:
def boss_turn(state):
    new_state = apply_ongoing(state)
    if new_state['boss_hp'] > 0:
        new_state['pc_hp'] -= max(new_state['boss_damage'] - new_state['armour'], 1)
    return new_state

In [67]:
def apply_ongoing(state):
    new_state = copy.deepcopy(state)
    new_state['armour'] = 0
    new_state['ongoing'] = []
    for status in state['ongoing']:
        for k in status[0]:
            new_state[k] += status[0][k]
        if len(status) > 1:
            new_state['ongoing'] += [status[1:]]
    return new_state

In [160]:
boss_turn(s3)

{'armour': 7,
 'boss_damage': 9,
 'boss_hp': 55,
 'cast': ['Shield', 'Poison'],
 'mana': 214,
 'ongoing': [[{'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7},
   {'armour': 7}],
  [{'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3}]],
 'pc_hp': 48,
 'spent': 286}

In [161]:
test_state_1 = {'pc_hp': 10, 'mana': 250, 'boss_hp': 13, 'boss_damage': 8, 'ongoing': [], 'spent': 0, 'cast': []}
s2 = cast_spell([s for s in spells if s['name'] == 'Poison'][0], test_state_1)
s3 = boss_turn(s2)
s3

{'armour': 0,
 'boss_damage': 8,
 'boss_hp': 10,
 'cast': ['Poison'],
 'mana': 77,
 'ongoing': [[{'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3}]],
 'pc_hp': 2,
 'spent': 173}

In [162]:
s4 = apply_ongoing(s3)
s5 = cast_spell([s for s in spells if s['name'] == 'Magic missile'][0], s4)
s6 = boss_turn(s5)
s6

{'armour': 0,
 'boss_damage': 8,
 'boss_hp': 0,
 'cast': ['Poison', 'Magic missile'],
 'mana': 24,
 'ongoing': [[{'boss_hp': -3}, {'boss_hp': -3}, {'boss_hp': -3}]],
 'pc_hp': 2,
 'spent': 226}

In [163]:
test_state_2 = {'pc_hp': 10, 'mana': 250, 'boss_hp': 14, 'boss_damage': 8, 'ongoing': [], 'spent': 0, 'cast': []}
s2 = cast_spell([s for s in spells if s['name'] == 'Recharge'][0], test_state_2)
s3 = boss_turn(s2)
s3

{'armour': 0,
 'boss_damage': 8,
 'boss_hp': 14,
 'cast': ['Recharge'],
 'mana': 122,
 'ongoing': [[{'mana': 101}, {'mana': 101}, {'mana': 101}, {'mana': 101}]],
 'pc_hp': 2,
 'spent': 229}

In [164]:
s4 = apply_ongoing(s3)
s5 = cast_spell([s for s in spells if s['name'] == 'Shield'][0], s4)
s6 = boss_turn(s5)
s6

{'armour': 7,
 'boss_damage': 8,
 'boss_hp': 14,
 'cast': ['Recharge', 'Shield'],
 'mana': 211,
 'ongoing': [[{'mana': 101}, {'mana': 101}],
  [{'armour': 7}, {'armour': 7}, {'armour': 7}, {'armour': 7}, {'armour': 7}]],
 'pc_hp': 1,
 'spent': 342}

In [165]:
s7 = apply_ongoing(s6)
s8 = cast_spell([s for s in spells if s['name'] == 'Drain'][0], s7)
s9 = boss_turn(s8)
s9

{'armour': 7,
 'boss_damage': 8,
 'boss_hp': 12,
 'cast': ['Recharge', 'Shield', 'Drain'],
 'mana': 340,
 'ongoing': [[{'armour': 7}, {'armour': 7}, {'armour': 7}]],
 'pc_hp': 2,
 'spent': 415}

In [166]:
s10 = apply_ongoing(s9)
s11 = cast_spell([s for s in spells if s['name'] == 'Poison'][0], s10)
s12 = boss_turn(s11)
s12

{'armour': 7,
 'boss_damage': 8,
 'boss_hp': 9,
 'cast': ['Recharge', 'Shield', 'Drain', 'Poison'],
 'mana': 167,
 'ongoing': [[{'armour': 7}],
  [{'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3},
   {'boss_hp': -3}]],
 'pc_hp': 1,
 'spent': 588}

In [167]:
s13 = apply_ongoing(s12)
s14 = cast_spell([s for s in spells if s['name'] == 'Magic missile'][0], s13)
s15 = boss_turn(s14)
s15

{'armour': 0,
 'boss_damage': 8,
 'boss_hp': -1,
 'cast': ['Recharge', 'Shield', 'Drain', 'Poison', 'Magic missile'],
 'mana': 114,
 'ongoing': [[{'boss_hp': -3}, {'boss_hp': -3}, {'boss_hp': -3}]],
 'pc_hp': 1,
 'spent': 641}

In [90]:
def finished(state):
    return state['boss_hp'] <= 0 or state['pc_hp'] <= 0

def victory(state):
    return finished(state) and state['pc_hp'] > 0

def defeat(state):
    return finished(state) and state['pc_hp'] <= 0

In [125]:
def ahistoric(state):
    return {k: state[k] for k in state if k != 'cast'}

In [186]:
agenda = [initial_state]
closed = []
while agenda:
        current_state = agenda[0]
        new_states = []
        if ahistoric(current_state) not in closed:
            closed += [ahistoric(current_state)]
            # print(current_state)
            if victory(current_state):
                # return current_state
                break
            for spell in valid_spells(spells, current_state):
                s2 = cast_spell(spell, current_state)
                if victory(s2):
                    new_states += [s2]
                else:
                    s3 = boss_turn(s2)
                    if victory(s3):
                        new_states += [s3]
                    if not finished(s3):
                        new_states += [apply_ongoing(s3)]
        # print(new_states)
        states_to_add = [s for s in new_states 
                         if ahistoric(s) not in closed
                         if len(s['cast']) <= 50]
        agenda = sorted(states_to_add + agenda[1:], key=lambda s: s['spent'])
        # agenda = new_states + agenda[1:]
current_state

{'armour': 7,
 'boss_damage': 9,
 'boss_hp': 0,
 'cast': ['Poison',
  'Recharge',
  'Drain',
  'Poison',
  'Recharge',
  'Shield',
  'Poison',
  'Magic missile',
  'Magic missile'],
 'mana': 241,
 'ongoing': [[{'boss_hp': -3}, {'boss_hp': -3}]],
 'pc_hp': 1,
 'spent': 1269}

#Part 2

In [182]:
def player_bleed(state):
    new_state = copy.deepcopy(state)
    new_state['pc_hp'] -= 1
    return new_state

In [185]:
agenda = [initial_state]
closed = []
while agenda:
        current_state = agenda[0]
        new_states = []
        if ahistoric(current_state) not in closed:
            closed += [ahistoric(current_state)]
            # print(current_state)
            if victory(current_state):
                # return current_state
                break
            for spell in valid_spells(spells, current_state):
                s2 = cast_spell(spell, current_state)
                if victory(s2):
                    new_states += [s2]
                else:
                    s3 = boss_turn(s2)
                    if victory(s3):
                        new_states += [s3]
                    s4 = player_bleed(s3)
                    if not finished(s4):
                        new_states += [apply_ongoing(s4)]
        # print(new_states)
        states_to_add = [s for s in new_states 
                         if ahistoric(s) not in closed
                         if len(s['cast']) <= 50]
        agenda = sorted(states_to_add + agenda[1:], key=lambda s: s['spent'])
        # agenda = new_states + agenda[1:]
current_state

{'armour': 0,
 'boss_damage': 9,
 'boss_hp': -1,
 'cast': ['Poison',
  'Recharge',
  'Shield',
  'Poison',
  'Recharge',
  'Shield',
  'Poison',
  'Magic missile',
  'Magic missile'],
 'mana': 201,
 'ongoing': [[{'boss_hp': -3}]],
 'pc_hp': 12,
 'spent': 1309}